In [20]:
import glob
import csv
import os
import re




In [21]:
def parse_each_file(file):
    useful_titles = ['DESCRIPTION OF NONCOMPLIANCE:','DESCRIPTIONS OF NONCOMPLIANCE','DESCRIPTION OF ACTIVITY OR OMISSION CONSTITUTING NONCOMPLIANCE:',
                 'DESCRIPTION OF ACTIVITY (OR OMISSION) IN NONCOMPLIANCE:','DESCRIPTION OF ACTIVITY CONSTITUTING NONCOMPLIANCE:',
                'DESCRIPTION OF ACTIVITY NONCOMPLIANCE:']

    with open(file, 'rt', encoding="utf8") as f:
        # Name Processing
        name = os.path.basename(f.name).strip(".txt")
        lines = f.readlines()       
        #circumstance
        cicumstance_list = []
        title_line = []
        for line in lines:
            for title in useful_titles:
                if title in line:
                    title_line.append(lines.index(line))
                    break            
        title_line = list(set(title_line))
            #for i in range(len(title_line)) :
                #print(lines[title_line[i]])
        for i in range (len(title_line )):#标题行数
            cicums_star = title_line[i]#定义circumstance start 是行数 数字      
            for j in range(cicums_star+1,len(lines),1): #标题下一行到结束
                line_info = lines[j]#每一行内容
                no_num = 0
                for i in range(len(line_info)):  #遍历第j行所有字符      
                    if line_info[i].isupper() == False: #判断非大写
                        no_num = no_num+ 1 #第j行非大写字符符号个数总和
                if 2<no_num <=10: #第j行非大写小于等于10个
                    #print("The Circumstance is ") #判断为标题
                    #cicums_stops = j #此时的第j行为结束
                    break
        
            for l in range(cicums_star+1,j):#输出结果 是个list
                cicumstance_list.append(lines[l])
        cir = "".join(cicumstance_list) #list转string 
        
        #citation
        #find citation in circumstance paragraph
        citation_list1 = []
        for line in cicumstance_list:    
            if "40." in line: 
                citation_star = line.find('40.')
                citation_fin = citation_star+7
                citation_name = line[citation_star:citation_fin]
                citation_list1.append(citation_name)
        citation_lists = list(set(citation_list1))#citation_result is a list
        #if no citation in circumstance paragraph
        flag2 = 0
        if len(citation_lists) == 0:
            for line in lines:
                if (line.find("DESCRIPTION OF RE")!=-1 or
                    line.find("REQUIREMENT(S) NOT COMPLIED") != -1):
                    flag2 = 1
                    continue
                elif (line.startswith("DISCRIPTION OF ACTION TO BE TAKEN")):
                    flag2 = 0
                if flag2 == 1:
                    
                    find_citation1 = re.findall(r"\d\d\.\d\d\d\d(?:\:|\;)", line)
                    find_citation2 = re.findall(r"\d\d\,\d\d\d\d:", line)
                    find_citation3 = re.findall(r"\d\d\. \d\d\d\d:", line)
                    find_citation4 = re.findall(r"\d\d\.\d\d\d\d\(", line)
                    find_citation5 = re.findall(r"\d\d\. \d\d\d\d\(", line)
                    
                    if len(find_citation1) != 0:
                        citation_lists += list(map(lambda x: x[:-1], find_citation1))
                    elif len(find_citation2) != 0:
                        citation_lists += list(map(lambda x: x[:2]+'.'+x[4:-1], find_citation2))
                    elif len(find_citation3) != 0:
                        citation_lists += list(map(lambda x: x[:3]+x[4:-1], find_citation3))
                    elif len(find_citation4) != 0:
                        citation_lists += list(map(lambda x: x[:-1], find_citation4))
                    elif len(find_citation5) != 0:
                        citation_lists += list(map(lambda x: x[:3]+x[4:-1], find_citation5))                   
        #find citation in whole files
        citation_list2 = []
        if len(citation_lists) == 0:
            for line in lines:
                if "40." in line: 
                    citation_star = line.find('40.')
                    citation_fin = citation_star+7
                    citation_name = line[citation_star:citation_fin]
                    citation_list2.append(citation_name)    
            citation_lists = list(set(citation_list2))
    return name,citation_lists,cir



In [22]:
def produce_csv(folder_path, output_filename):
    """Produces a csv of file name, citations and descriptions of a folder with text files
    Parameters
    ----------
    folder_path : str
        String representation of the folder path containing the text files.
    output_filename : str
        Name of the output file the merged lines will be written to.
    """
    # make sure there's a slash to the folder path
    folder_path += "" if folder_path[-1] == "/" else "/"
    # get all text files
    txt_files = glob.glob(folder_path + "*.txt")

    # write to file
    csvex = csv.writer(open(folder_path + output_filename, "wt"))
    headers = ['file name', 'citation', 'description']
    csvex.writerow(headers)

    for f in txt_files:
        s = parse_each_file(f)
        #print(s)
        csvex.writerow(s)


In [23]:
produce_csv('/Users/huangzhiyi/Desktop/GQP/dataprocess/NONs_txt_split/DESCRIPTION OF NONCOMPLIANCE_1', 'out_NONCOMPLIANCE_1.csv')
